<h3>1) Load and Processing</h3>
<p>Importing data with pandas into a DF, converting date types, detection of duplicates and missing values, outliers.</p>

In [2]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go


/var/folders/bc/v182gz697ql0f2t1c5xkvdn00000gn/T/ipykernel_963/1745800605.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
car_sales = pd.read_csv("car_sales_data.csv")
car_sales.info()
car_sales.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500000 entries, 0 to 2499999
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Date               object 
 1   Salesperson        object 
 2   Customer Name      object 
 3   Car Make           object 
 4   Car Model          object 
 5   Car Year           int64  
 6   Sale Price         int64  
 7   Commission Rate    float64
 8   Commission Earned  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 171.7+ MB


,Date,Salesperson,Customer Name,Car Make,Car Model,Car Year,Sale Price,Commission Rate,Commission Earned
0,2022-08-01,Monica Moore MD,Mary Butler,Nissan,Altima,2018,15983,0.070495,1126.73
1,2023-03-15,Roberto Rose,Richard Pierce,Nissan,F-150,2016,38474,0.134439,5172.40
2,2023-04-29,Ashley Ramos,Sandra Moore,Ford,Civic,2016,33340,0.114536,3818.63
3,2022-09-04,Patrick Harris,Johnny Scott,Ford,Altima,2013,41937,0.092191,3866.20
4,2022-06-16,Eric Lopez,Vanessa Jones,Honda,Silverado,2022,20256,0.113490,2298.85


In [4]:
# Converting Date column to datetime
car_sales["Date"] = pd.to_datetime(car_sales['Date'])

# Searching for duplicate records
duplicates = car_sales.duplicated(subset = ['Date', 'Salesperson', 'Customer Name', 'Car Model', 'Sale Price']).any()
print(duplicates) ## False if not duplicates

# Looking for null values
null_values = car_sales.isnull().sum()
print(null_values)


False
Date                 0
Salesperson          0
Customer Name        0
Car Make             0
Car Model            0
Car Year             0
Sale Price           0
Commission Rate      0
Commission Earned    0
dtype: int64


Removing Outliers 

For this we delete sales significantly outside the Interquartile range (IQR)

In [5]:
Q1 = car_sales['Sale Price'].quantile(0.25)
Q3 = car_sales['Sale Price'].quantile(0.75)
IQR = Q3 - Q1

# Significantly outside bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_below = car_sales[car_sales['Sale Price'] < lower_bound]
print(outliers_below)
outliers_above = car_sales[car_sales['Sale Price'] > upper_bound]
print(outliers_above)
## No outliers found

Empty DataFrame
Columns: [Date, Salesperson, Customer Name, Car Make, Car Model, Car Year, Sale Price, Commission Rate, Commission Earned]
Index: []
Empty DataFrame
Columns: [Date, Salesperson, Customer Name, Car Make, Car Model, Car Year, Sale Price, Commission Rate, Commission Earned]
Index: []


<h4>2. Salespersons Performance Analysis</h4>
<p>Display key metrics including total sales, average sale price, total commissions, and performance rankings. This analysis highlights top performers and pinpoints improvement opportunities.</p>


In [6]:
#Total sales and average sale price per salesperson
sales_performance = car_sales.groupby('Salesperson').agg(
  TotalSales = pd.NamedAgg(column= 'Sale Price', aggfunc= 'sum'),
  AverageSalePrice = pd.NamedAgg(column= 'Sale Price', aggfunc= 'mean'),
  TotalComissions = pd.NamedAgg (column = 'Commission Earned', aggfunc= 'sum')
).reset_index()

#Rank salespersons based on TotalSales
sales_performance['SalesRank'] = sales_performance['TotalSales'].rank(method='dense', ascending=False)
# on TotalComissions
sales_performance['Comission_Rank'] = sales_performance['TotalComissions'].rank(method='dense', ascending=False)

#Show in pandas
#sales_performance.sort_values(by=['SalesRank', 'Comission_Rank'], ascending = True)

## Visualization Plotly
# Top 10 Salesperson
N = 10
top_salespersons = sales_performance.sort_values(by='TotalSales', ascending=False).head(N)

fig = px.bar(top_salespersons,
             x='Salesperson',
             y=['TotalSales', 'TotalComissions'],
             title='Total Sales and Commissions revenue for Top 10 Salespersons',
             labels={'value':'Amount', 'variable':'Metric'},
             barmode='group')
fig.update_layout(xaxis_title='Salesperson', yaxis_title='Amount', xaxis={'categoryorder':'total descending'})
fig.show()

# Average Sale Price of Top Performers
fig_avg = px.bar(top_salespersons,
                 x='Salesperson',
                 y='AverageSalePrice',
                 title='Average Sale Price for Top 10 Salespersons',
                 labels={'AverageSalePrice': 'Average Sale Price'},
                 text='AverageSalePrice')
fig_avg.update_layout(xaxis_title='Salesperson', yaxis_title='Average Sale Price ($)',
                      xaxis={'categoryorder':'total descending'},
                      yaxis=dict(type='log'))  # Optional: Use a log scale if wide variance
fig_avg.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_avg.show()



<h4>3. Inventory Insights: Identifying Top-Selling Car Models </h4>
<p>Analyze sales by car model and year to identify the most popular options.</p>

In [7]:

# Group by car make and year, and count the number of sales for each combination
sales_counts = car_sales.groupby(['Car Model', 'Car Year']).size().reset_index(name='Sales Count')

# Sort from highest Sales Count 
top_sales_count= sales_counts.sort_values(by = 'Sales Count', ascending=False).head(10)
#Print
#top_sales_count

#Visualization Ploty
# Create a table 
fig = ff.create_table(top_sales_count)

# Update table layout for a cleaner look
fig.update_layout(
    title_text='Top 10 Car Model-Year Combinations by Sales Count',
    title_x=0.5,  # Center the title
    margin=dict(l=5, r=5, t=30, b=5)  # Adjust margins to fit title and ensure table fits in the layout
)

fig.show()


<h4>4. Most Popular Car Make</h4>

In [11]:
# Aggregate data to count sales per car make
car_make_counts = car_sales['Car Make'].value_counts().reset_index()
car_make_counts.columns = ['Car Make', 'Sales Count']



# Visualize the data with a donut chart
fig = px.pie(car_make_counts, values='Sales Count', names='Car Make',
             title="Sales Distribution by Car Make",
             hole=0.5)  # The 'hole' parameter creates the donut chart appearance

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

<h4>5. Sales Trends and Seasonality</h4>
<p> How do car sales trend over time, and is there any seasonality?</p>

In [ ]:

# Extract year and month from 'Date'
car_sales['Year'] = car_sales['Date'].dt.year
car_sales['Month'] = car_sales['Date'].dt.month

#Group by year and month, then count sales
sales_trends = car_sales.groupby(['Year', 'Month']).size().reset_index(name='Sales Count')

# Sort the DataFrame for chronological plotting
sales_trends = sales_trends.sort_values(by=['Year', 'Month'])

# Create a line plot of sales trends
fig = px.line(sales_trends, x=pd.to_datetime(sales_trends[['Year', 'Month']].assign(DAY=1)), y='Sales Count',
              title='Monthly Car Sales Trends',
              labels={'x': 'Date', 'Sales Count': 'Number of Sales'})

# Enhance layout
fig.update_layout(xaxis_title='Date',
                  yaxis_title='Number of Sales',
                  xaxis=dict(tickformat='%Y-%m'),
                  hovermode='x')

fig.show()